In [1]:
from desc.objectives import ParticleTracer, ObjectiveFunction
from desc.examples import get
from desc.grid import Grid
import desc.io
from desc.backend import jnp
import matplotlib.pyplot as plt
import numpy as np

DESC version 0+untagged.5549.gb032f77.dirty,using JAX backend, jax version=0.4.14, jaxlib version=0.4.14, dtype=float64
Using device: CPU, with 2.32 GB available memory


In [2]:
eq = desc.io.load("test_run.h5")
eq._iota = eq.get_profile("iota").to_powerseries(order=eq.L, sym=True)
eq._current = None
# eq.solve()

In [3]:
tmin = 0
tmax = 0.00007
nt = 20
time = jnp.linspace(tmin, tmax, nt)
psi_i = 0.7
theta_i = 0.2
zeta_i = 0.2

mass = 1.673e-27
Energy = 3.52e6*1.6e-19
ini_vpar = 0.5*jnp.sqrt(2*Energy/mass)
ini_cond = [psi_i, theta_i, zeta_i, float(ini_vpar)]

mass_charge = mass/1.6e-19

grid = Grid(jnp.array([jnp.sqrt(psi_i), theta_i, zeta_i]).T, jitable=True, sort=False)
data = eq.compute("|B|", grid=grid)

mu = Energy/(mass*data["|B|"]) - (ini_vpar**2)/(2*data["|B|"])

ini_param = [float(mu), mass_charge]


objective = ParticleTracer(eq=eq, output_time=time, initial_conditions=ini_cond, initial_parameters=ini_param)

In [4]:
objective.build()
solution = objective.compute(*objective.xs(eq))
print(solution)

2023-09-13 00:20:18.612270: E external/xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module jit__odeint_wrapper] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2023-09-13 00:22:48.430204: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 4m29.821270294s

********************************
[Compiling module jit__odeint_wrapper] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


0.5980447886775762


In [5]:
objective = ParticleTracer(eq=eq, output_time=time, initial_conditions=ini_cond, initial_parameters=ini_param)
ObjFunction = ObjectiveFunction([objective])
ObjFunction.build()

Building objective: Particle Tracer


In [6]:
ObjFunction.compile(mode="bfgs")

Compiling objective function and derivatives: ['Particle Tracer']


2023-09-13 00:30:24.350217: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 4m31.94021056s

********************************
[Compiling module jit_compute_scalar] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


: 

In [ ]:
ObjFunction.grad(ObjFunction.x(eq))
#print(ObjFunction.x(eq))
objective.xs(eq)

(array([ 3.82170567e-05,  5.78816448e-04,  1.27507389e-05, -1.55089996e-04,
        -6.28066446e-06, -6.28203127e-05, -2.30395901e-05, -1.46837402e-04,
         7.63749783e-05,  2.72564948e-07,  6.44465446e-06, -3.68993822e-05,
        -2.54602327e-07,  1.37587658e-05,  2.18221238e-05, -2.13573445e-05,
        -3.77009519e-08,  1.80910854e-06,  3.84326513e-05,  3.31470145e-05,
         2.14637665e-04,  3.15078961e-03, -9.02670870e-04,  4.29266601e-04,
         1.66942659e-04, -3.43670743e-03,  8.71806701e-05,  9.83954929e-04,
        -5.81068885e-04,  4.74802587e-06, -2.39785187e-04,  1.57984598e-04,
         3.23948370e-07, -8.84892044e-05,  4.48370750e-06,  7.51548025e-05,
        -2.16185457e-07, -4.67003227e-06,  6.28705045e-05, -6.39309851e-05,
        -2.85879908e-03,  5.92815511e-05,  2.12974151e-04,  2.66009234e-03,
        -2.76560985e-04,  3.31163308e-04,  1.85510286e-04, -8.26017921e-04,
         1.97635704e-04, -5.23086523e-06,  5.30351506e-04, -3.97953112e-04,
        -5.1

In [ ]:
objective.args

['R_lmn', 'Z_lmn', 'L_lmn', 'i_l', 'c_l', 'Psi']